In [1]:
# The basics
import numpy as np
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Softmax, Conv1D, MaxPooling1D, GlobalMaxPooling1D

# utils
import os
import subprocess

In [2]:
parent_relative_path = ".."

In [3]:
colab = 'google.colab' in str(get_ipython())

In [4]:
if colab:
    print("Colab babe")
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

    os.system('cp "/content/gdrive/My Drive/Colab Notebooks/.kaggle/kaggle.json" /root/.kaggle/kaggle.json')
    subprocess.run(["git", "clone", "https://github.com/codefupanda/customer_interaction_summary.git"])
    os.system('cd customer_interaction_summary && make requirements && make data > logs.logs')
    parent_relative_path = "./customer_interaction_summary"

In [5]:
isear = pd.read_csv(parent_relative_path + '/data/raw/isear.csv', sep='|', error_bad_lines=False, usecols=['Field1', 'SIT', 'EMOT'])

In [6]:
number_of_classes = len(isear.EMOT.unique())

In [7]:
maxlen = 1000
max_words = 10000

In [8]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(isear['SIT'])
sequences = tokenizer.texts_to_sequences(isear['SIT'])

In [9]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9063 unique tokens.


In [10]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post')

In [11]:
x_train, x_test, y_train, y_test = train_test_split(data, isear['EMOT'])

## Model creation time

In [12]:
model = Sequential()
model.add(Embedding(max_words, output_dim=50, input_length=maxlen))
model.add(Conv1D(32, 7, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(32, 7, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(number_of_classes + 1,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 50)          500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 994, 32)           11232     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 198, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 192, 32)           7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 264       
Total params: 518,696
Trainable params: 518,696
Non-trainable params: 0
__________________________________________________

In [16]:
model.fit(x_train, to_categorical(y_train),
          epochs=5,
          batch_size=32,
          validation_data=(x_test, to_categorical(y_test)))

Train on 5749 samples, validate on 1917 samples
Epoch 1/5
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.4413 - accuracy: 0.8690 - val_loss: 1.6903 - val_accuracy: 0.5081
Epoch 2/5
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.3044 - accuracy: 0.9222 - val_loss: 1.8993 - val_accuracy: 0.5003
Epoch 3/5
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.2115 - accuracy: 0.9494 - val_loss: 2.0981 - val_accuracy: 0.5013
Epoch 4/5
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.1484 - accuracy: 0.9643 - val_loss: 2.2648 - val_accuracy: 0.4888
Epoch 5/5
5749/5749 [==============================] - 11s 2ms/sample - loss: 0.1087 - accuracy: 0.9751 - val_loss: 2.4301 - val_accuracy: 0.4825


In [14]:
y_pred = model.predict_classes(x_test)
y_pred

array([4, 5, 4, ..., 1, 7, 7])

In [15]:
confusion_matrix(y_test, y_pred)

array([[200,  13,  18,  12,   6,  20,  17],
       [ 14, 161,   6,  17,  32,  11,  14],
       [  7,  12, 110,  18,  53,  17,  49],
       [ 25,  19,  28, 147,  12,  20,  30],
       [ 13,  23,  46,  15, 135,  11,  18],
       [ 24,  16,  31,  21,  39,  87,  75],
       [ 21,  14,  48,  13,  15,  27, 137]])